# Code for transferring LowRedux Arc solution to PYPIT

    Generate hdf5 files

In [124]:
# imports
import h5py
from scipy.io.idl import readsav
import test_holy as tstholy
import parse_lowrdx_arcs as plowa

## LRISb 600

### Read

In [114]:
# IDL save file
sav_file = os.getenv('LONGSLIT_DIR')+'calib/linelists/lris_blue_600.sav'
s = readsav(sav_file)
ctbl = Table(s['calib'])  # For writing later

In [49]:
outfil = 'test_arcs/LRISb_600.hdf5'

In [17]:
# Line list
lamps=['ZnI', 'CdI', 'HgI']
air_alist = tstholy.ararclines.load_arcline_list(None,None, lamps, None, use_vac=False)
alist = tstholy.ararclines.load_arcline_list(None,None, lamps, None)

[INFO]    :: Rejecting select ZnI lines
[INFO]    :: Rejecting select CdI lines
[INFO]    :: Rejecting select HgI lines
[INFO]    :: Rejecting select ZnI lines
[INFO]    :: Rejecting select CdI lines
[INFO]    :: Rejecting select HgI lines


### Init

In [5]:
ss = 0

In [125]:
odict = dict(npix=len(s['archive_arc'][0]), lamps=np.array(lamps), nspec=len(s['archive_arc']),infil=sav_file, IDairvac='vac')
odict

{'IDairvac': 'vac',
 'infil': '/u/xavier/idl/xidl/Spec/Longslit/calib/linelists/lris_blue_600.sav',
 'lamps': array(['ZnI', 'CdI', 'HgI'], 
       dtype='|S3'),
 'npix': 2048,
 'nspec': 25}

### Find peaks

In [7]:
spec = s['archive_arc'][ss]

In [8]:
tampl, tcent, twid, w, yprep = plowa.find_peaks(spec)
pixpk  = tcent[w]
pixampl = tampl[w]

In [27]:
pixpk

array([    6.96347629,   259.05697654,   308.03278559,   505.33123154,
         560.35825722,   779.81521594,   788.07340425,   849.88349094,
         884.64830427,   897.12069187,  1094.2832287 ,  1139.56636245,
        1225.33163475,  1266.12438555,  1299.04532828,  1311.31459385,
        1364.42805331,  1389.75112081,  1703.98837067,  1714.42498088,
        1737.07062389,  1747.08800529,  1869.39984907,  1923.49732366,
        1974.48221379,  2011.49204726,  2029.65877979])

In [10]:
# Check
if False:
    plt.clf()
    ax = plt.gca()
    ax.plot(np.arange(odict['npix']), yprep, 'k', drawstyle='mid-steps')
    ax.scatter(pixpk, pixampl, marker='o')
    plt.show()

### ID lines

In [12]:
# Wavelength solution
wv_air = plowa.cheby_val(s['calib'][ss]['ffit'], np.arange(odict['npix']),
                   s['calib'][ss]['nrm'],s['calib'][ss]['nord'])
#xdb.xplot(wv_air, spec)

In [38]:
# Peaks
twave_air = plowa.cheby_val(s['calib'][ss]['ffit'], pixpk,
                   s['calib'][ss]['nrm'],s['calib'][ss]['nord'])
#twave_air

In [24]:
# Vacuum
reload(tstholy)
twave_vac = tstholy.arwave.airtovac(twave_air*u.AA)
twave_vac

<Quantity [ 5771.41299793, 5462.32824543, 5401.74959892, 5156.25398846,
            5087.43077381, 4811.90970678, 4801.51972551, 4723.72966188,
            4679.96632699, 4664.26522587, 4416.29011305, 4359.4706136 ,
            4252.09617148, 4201.16227744, 4160.13339035, 4144.86131466,
            4078.87832267, 4047.49945186, 3663.87508273, 3651.36051886,
            3624.26621276, 3612.30778317, 3467.72639073, 3404.69777917,
            3345.86401496, 3303.52283322, 3282.85666277] Angstrom>

In [37]:
tmp = np.array(air_alist['wave']) # Air
tmp.sort()
#tmp

In [19]:
tmp2 = np.array(alist['wave']) # Vacuum
tmp2.sort()
tmp2

array([ 3019.24  ,  3022.384 ,  3036.66  ,  3072.95  ,  3076.79  ,
        3081.7172,  3083.4887,  3126.58  ,  3134.0746,  3253.4622,
        3261.9951,  3283.28  ,  3303.53  ,  3345.98  ,  3404.6287,
        3467.1923,  3468.6477,  3500.9531,  3611.5375,  3651.198 ,
        3655.883 ,  3664.327 ,  3884.44  ,  3983.0519,  4047.708 ,
        4078.988 ,  4141.4665,  4307.8832,  4359.56  ,  4414.2288,
        4663.6572,  4679.4587,  4681.45  ,  4723.47  ,  4801.254 ,
        4811.87  ,  5087.2393,  5156.0964,  5183.42  ,  5462.268 ,
        5771.21  ,  5792.276 ,  6100.8305,  6113.187 ,  6326.9151,
        6331.7636,  6364.1   ,  6440.249 ,  6779.9865,  6909.37  ,
        7083.854 ,  7093.815 ,  7347.6941,  8202.5633,  9295.    ])

In [39]:
# ID
idwv = np.zeros_like(pixpk)
for kk,twv in enumerate(twave_vac.value):
    # diff
    diff = np.abs(twv-alist['wave'])
    if np.min(diff) < 0.6:  #  May change for different instruments
        idwv[kk] = alist['wave'][np.argmin(diff)]
#idwv

array([ 5771.21  ,  5462.268 ,     0.    ,  5156.0964,  5087.2393,
        4811.87  ,  4801.254 ,  4723.47  ,  4679.4587,     0.    ,
           0.    ,  4359.56  ,     0.    ,     0.    ,     0.    ,
           0.    ,  4078.988 ,  4047.708 ,  3664.327 ,  3651.198 ,
           0.    ,     0.    ,  3467.1923,  3404.6287,  3345.98  ,
        3303.53  ,  3283.28  ])

### Output

In [119]:
# open
outh5 = h5py.File(outfil, 'w')

In [120]:
# Meta data
outh5.create_group('meta')
for key in odict.keys():
    outh5['meta'][key] = odict[key]

In [121]:
# Spectra
outh5.create_group('arcs')
for ii in range(1):
    sii = str(ii)
    outh5['arcs'].create_group(sii)
    # Datasets
    outh5['arcs'][sii]['spec'] = spec
    outh5['arcs'][sii]['pixpk'] = pixpk
    outh5['arcs'][sii]['ID'] = idwv
    outh5['arcs'][sii]['ID'].attrs['airvac'] = 'vac'
    # LR wavelengths
    outh5['arcs'][sii]['LR_wave'] = wv_air
    outh5['arcs'][sii]['LR_wave'].attrs['airvac'] = 'air'
    # LR Fit
    outh5['arcs'][sii].create_group('LR_fit')
    for key in ctbl.keys():
        outh5['arcs'][sii]['LR_fit'][key] = ctbl[ii][key]

In [122]:
outh5['arcs']['0']['LR_fit'].keys()

[u'DISP_CEN',
 u'FFIT',
 u'FLG_REJ',
 u'FUNC',
 u'HSIG',
 u'LSIG',
 u'MAXREJ',
 u'MINPT',
 u'NITER',
 u'NORD',
 u'NRM',
 u'RMS',
 u'WAVE_CEN']

In [123]:
outh5.close()

### Test reading

In [135]:
tmp = h5py.File(outfil,'r')

In [136]:
tmp['meta'].keys()

[u'IDairvac', u'infil', u'lamps', u'npix', u'nspec']

In [137]:
mdict = {}
for key in tmp['meta'].keys():
    mdict[key] = tmp['meta'][key].value
mdict['lamps'] = list(mdict['lamps'])

In [138]:
mdict

{u'IDairvac': u'vac',
 u'infil': u'/u/xavier/idl/xidl/Spec/Longslit/calib/linelists/lris_blue_600.sav',
 u'lamps': ['ZnI', 'CdI', 'HgI'],
 u'npix': 2048,
 u'nspec': 25}

In [139]:
tmp['arcs']['0'].keys()

[u'ID', u'LR_fit', u'LR_wave', u'pixpk', u'spec']

In [134]:
tmp.close()

In [143]:
np.round(143,-2)

100

In [142]:
rstate.randint(3,6,10)

array([4, 5, 5, 5, 3, 3, 5, 5, 5, 3])